1. Exploratory Data Analysis 

My dataset is divided into the following columns:
"RegionID","RegionName","State","Metro","CountyName","SizeRank",
"1996-04","1996-05","1996-06","1996-07","1996-08","1996-09","1996-10","1996-11","1996-12",
"1997-01","1997-02","1997-03","1997-04","1997-05","1997-06","1997-07","1997-08","1997-09","1997-10","1997-11","1997-12",
"1998-01","1998-02","1998-03","1998-04","1998-05","1998-06","1998-07","1998-08","1998-09","1998-10","1998-11","1998-12",
"1999-01","1999-02","1999-03","1999-04","1999-05","1999-06","1999-07","1999-08","1999-09","1999-10","1999-11","1999-12",
"2000-01","2000-02","2000-03","2000-04","2000-05","2000-06","2000-07","2000-08","2000-09","2000-10","2000-11","2000-12",
"2001-01","2001-02","2001-03","2001-04","2001-05","2001-06","2001-07","2001-08","2001-09","2001-10","2001-11","2001-12",
"2002-01","2002-02","2002-03","2002-04","2002-05","2002-06","2002-07","2002-08","2002-09","2002-10","2002-11","2002-12",
"2003-01","2003-02","2003-03","2003-04","2003-05","2003-06","2003-07","2003-08","2003-09","2003-10","2003-11","2003-12",
"2004-01","2004-02","2004-03","2004-04","2004-05","2004-06","2004-07","2004-08","2004-09","2004-10","2004-11","2004-12",
"2005-01","2005-02","2005-03","2005-04","2005-05","2005-06","2005-07","2005-08","2005-09","2005-10","2005-11","2005-12",
"2006-01","2006-02","2006-03","2006-04","2006-05","2006-06","2006-07","2006-08","2006-09","2006-10","2006-11","2006-12",
"2007-01","2007-02","2007-03","2007-04","2007-05","2007-06","2007-07","2007-08","2007-09","2007-10","2007-11","2007-12",
"2008-01","2008-02","2008-03","2008-04","2008-05","2008-06","2008-07","2008-08","2008-09","2008-10","2008-11","2008-12",
"2009-01","2009-02","2009-03","2009-04","2009-05","2009-06","2009-07","2009-08","2009-09","2009-10","2009-11","2009-12",
"2010-01","2010-02","2010-03","2010-04","2010-05","2010-06","2010-07","2010-08","2010-09","2010-10","2010-11","2010-12",
"2011-01","2011-02","2011-03","2011-04","2011-05","2011-06","2011-07","2011-08","2011-09","2011-10","2011-11","2011-12",
"2012-01","2012-02","2012-03","2012-04","2012-05","2012-06","2012-07","2012-08","2012-09","2012-10","2012-11","2012-12",
"2013-01","2013-02","2013-03","2013-04","2013-05","2013-06","2013-07","2013-08","2013-09","2013-10","2013-11","2013-12",
"2014-01","2014-02","2014-03","2014-04","2014-05","2014-06","2014-07","2014-08","2014-09","2014-10","2014-11","2014-12",
"2015-01","2015-02","2015-03","2015-04","2015-05","2015-06","2015-07","2015-08","2015-09","2015-10","2015-11","2015-12",
"2016-01","2016-02","2016-03","2016-04","2016-05","2016-06","2016-07","2016-08"

RegionID is a numerical identifier with no real qualitative purpose for this experiment and will be dropped in data preprocessing. RegionName is the label for the town or region, State is the state the region is located, Metro is the nearest metropolitan area to the region, and CountryName is the country the region resides in, all these columns will stay in our dataframe. The next field is SizeRank, whish a numerical label used to define the size of an estate. all of the following columns represent the median apprasal rate per month in the designated region, for the purposes of this experiment i will be dropping the years 1996 and 2016 as their data is incomplete for those years.

2. Baseline method implementation and result analysis 


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
import json
import lightgbm as lgbm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [13]:
df = pd.read_csv('./City_Zhvi_AllHomes.csv')

df.head(10)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,18959,Las Vegas,NV,Las Vegas,Clark,6,121600.0,120900.0,120400.0,120300.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,54296,San Diego,CA,San Diego,San Diego,7,161100.0,160700.0,160400.0,160100.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,38128,Dallas,TX,Dallas-Fort Worth,Dallas,8,NaN,NaN,NaN,NaN,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,33839,San Jose,CA,San Jose,Santa Clara,9,224500.0,224900.0,225400.0,226100.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,25290,Jacksonville,FL,Jacksonville,Duval,10,77500.0,77200.0,76800.0,76600.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [14]:
df.shape

(10730, 251)

In [15]:
df.describe()

,RegionID,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
count,10730.000000,10730.000000,8.083000e+03,8.179000e+03,8.218000e+03,8.221000e+03,8.229000e+03,8.229000e+03,8.229000e+03,8.235000e+03,...,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04,1.073000e+04
mean,87338.218080,5365.500000,1.203738e+05,1.204542e+05,1.203388e+05,1.204008e+05,1.204700e+05,1.205682e+05,1.207127e+05,1.210040e+05,...,2.363926e+05,2.372042e+05,2.378533e+05,2.386053e+05,2.395900e+05,2.405300e+05,2.413038e+05,2.419801e+05,2.428608e+05,2.436931e+05
std,128121.389608,3097.628528,7.835705e+04,7.790580e+04,7.785422e+04,7.795274e+04,7.795952e+04,7.802532e+04,7.816369e+04,7.835491e+04,...,2.423569e+05,2.432497e+05,2.441329e+05,2.451509e+05,2.461725e+05,2.467904e+05,2.471556e+05,2.473850e+05,2.477199e+05,2.479585e+05
min,3304.000000,1.000000,2.400000e+04,2.430000e+04,2.420000e+04,2.370000e+04,2.350000e+04,2.340000e+04,2.380000e+04,2.430000e+04,...,3.090000e+04,3.050000e+04,3.020000e+04,3.040000e+04,3.100000e+04,3.160000e+04,3.200000e+04,3.230000e+04,3.300000e+04,3.360000e+04
25%,19606.250000,2683.250000,7.520000e+04,7.570000e+04,7.560000e+04,7.560000e+04,7.580000e+04,7.580000e+04,7.570000e+04,7.590000e+04,...,1.213000e+05,1.218000e+05,1.219000e+05,1.221250e+05,1.224000e+05,1.228000e+05,1.232000e+05,1.235000e+05,1.239250e+05,1.244000e+05
50%,35970.000000,5365.500000,1.037000e+05,1.041000e+05,1.038000e+05,1.040000e+05,1.042000e+05,1.046000e+05,1.047000e+05,1.048000e+05,...,1.727000e+05,1.733000e+05,1.735000e+05,1.742000e+05,1.750500e+05,1.757500e+05,1.762000e+05,1.766000e+05,1.774000e+05,1.781000e+05
75%,53160.250000,8047.750000,1.420000e+05,1.421000e+05,1.418750e+05,1.420000e+05,1.419000e+05,1.419000e+05,1.421000e+05,1.423500e+05,...,2.669750e+05,2.679000e+05,2.688000e+05,2.699750e+05,2.708750e+05,2.724000e+05,2.734000e+05,2.738000e+05,2.750750e+05,2.762750e+05
max,737791.000000,10730.000000,2.574500e+06,2.553100e+06,2.541500e+06,2.531700e+06,2.526600e+06,2.530400e+06,2.535100e+06,2.543800e+06,...,6.054000e+06,6.081000e+06,6.128400e+06,6.182400e+06,6.230600e+06,6.263700e+06,6.266700e+06,6.235800e+06,6.213000e+06,6.200200e+06


In [16]:
#preprocess and clean data
# check if there is NaN in the dataset
print( df.isnull().sum() )

#Drop NaNs if there is any
df = df.dropna()

RegionID        0
RegionName      0
State           0
Metro         471
CountyName      0
             ... 
2016-04         0
2016-05         0
2016-06         0
2016-07         0
2016-08         0
Length: 251, dtype: int64


In [17]:
#remove unnecessary data

cols_to_drop = [ 'RegionID', '1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09', '1996-10', 
                '1996-11', '1996-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08']

df = df.drop( columns=cols_to_drop )

In [18]:
df.shape

(7624, 233)

3. Proposal for next steps

